Calculate Soil Moisture

Based on Sentinel Hub script: https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-1/soil_moisture_estimation/

In [ ]:
import numpy as np
import rasterio
from rasterio.enums import Resampling

In [ ]:
def read_geotiff(input_file):
    # Read the VV and VH bands from the GeoTIFF file
    with rasterio.open(input_file) as src:
        vv_band = src.read(1)  # VV band
        vh_band = src.read(2)  # VH band
        profile = src.profile
    return vv_band, vh_band, profile

def evaluate_pixel(vv_band, samples):
    count = len(samples)

    max_val = max(samples)
    min_val = min(samples)
    sum_VV = sum(samples)

    # Calculate the sensitivity
    sensitivity = max_val - min_val

    # Calculate the Soil Moisture (Mv)
    Mv = (vv_band - min_val) / sensitivity
    print(Mv)

    return Mv

def apply_colormap(soil_moisture):
    vmin = 0
    vmax = 0.6
    diffv = vmax - vmin
    r, g, b = np.zeros_like(soil_moisture), np.zeros_like(soil_moisture), np.zeros_like(soil_moisture)

    T1, T2, T3, T4 = 0.1, 0.3, 0.4, 0.5
    Thresh_1 = vmin + T1 * diffv
    Thresh_2 = vmin + T2 * diffv
    Thresh_3 = vmin + T3 * diffv
    Thresh_4 = vmin + T4 * diffv

    for i in range(soil_moisture.shape[0]):
        for j in range(soil_moisture.shape[1]):
            v = soil_moisture[i, j]
            if v <= 0:
                r[i, j], g[i, j], b[i, j] = 1, 1, 1
            elif v < Thresh_1:
                r[i, j] = 0.5 + (v - vmin) / (Thresh_1 - vmin) / 2
            elif v < Thresh_2:
                r[i, j], g[i, j] = 1, (v - Thresh_1) / (Thresh_2 - Thresh_1)
            elif v < Thresh_3:
                r[i, j], g[i, j], b[i, j] = 1 + (Thresh_2 - v) / (Thresh_3 - Thresh_2), 1, (v - Thresh_2) / (Thresh_3 - Thresh_2)
            elif v < Thresh_4:
                r[i, j], g[i, j], b[i, j] = 0, 1 + (Thresh_3 - v) / (Thresh_4 - Thresh_3), 1
            else:
                b[i, j] = 1.0 + (Thresh_4 - v) / (vmax - Thresh_4) / 2

    return r, g, b

def save_geotiff(output_file, soil_moisture, profile, r, g, b):
    # Stack RGB channels to create a 3-band image
    output_data = np.stack((r, g, b), axis=-1)
    profile.update(count=3, dtype=rasterio.float32)
    
    with rasterio.open(output_file, 'w', **profile) as dst:
        for i in range(3):
            dst.write(output_data[:, :, i], i + 1)

def get_soil_moisture(input_file, output_file):
    vv_band, vh_band, profile = read_geotiff(input_file)

    # Process the VV band to calculate Soil Moisture (Mv)
    samples = vv_band.flatten()
    soil_moisture = evaluate_pixel(vv_band, samples)

    # Apply colormap to the soil moisture
    r, g, b = apply_colormap(soil_moisture)

    save_geotiff(output_file, soil_moisture, profile, r, g, b)

In [ ]:
input_file = "../data/S1A_IW_GRDH_1SDV_20250401_Clip.tif"
output_file = "../data/S1A_IW_GRDH_1SDV_20250401_soil_moisture.tif"
get_soil_moisture(input_file, output_file)